## EigenPulse Detecting Surges in Large Streaming Graphs with Row Augmentation
EigenPulse is a streaming algorithm to detect surges of sliding windows in real time.
### Abstract
How can we spot dense blocks in a large streaming graph efficiently? Anomalies such as fraudulent attacks, spamming, and DDoS attacks, can create dense blocks in a short time window, emerging a surge of density in a streaming graph. However, most existing methods detect dense blocks in a static graph or a snapshot of dynamic graphs, which need to inefficiently rerun the algorithms for a streaming graph. Moreover, some works on streaming graphs are either consuming much time on updating algorithm for every incoming edge, or spotting the whole snapshot of a graph instead of the attacking sub-block. Therefore, we propose a row-augmented matrix with sliding window to model a streaming graph, and design the AugSVD algorithm for computation- and memory-efficient singular decomposition. EigenP ulse is then proposed to spot the density surges in streaming graphs based on the singular spectrum. We theoretically analyze the robustness of our method. Experiments on real datasets with injections show our perfor- mance and efficiency compared with the state-of-the-art baseline.

In [ ]:
import spartan as st

In [ ]:
f,idxtypes = st.loadTensorStream(filename="./inputData/beer.tensor",
                                 col_idx = [0,1,2,3], col_types=[int,int,int,int])

### Function loadTensorStream():
tensor_data.data has multiple-colum attributes, and a single-colum values (optional). The following table shows an example of 10000 three-tuple (user, object, time) and the 4th-colum is the frequency. 

col_idx: list of the columns to process

coltypes: list of data types of the columns to process

**Return**

**f:**: file iterator of input file

**idxtypes**: column idx-type dict

In [ ]:
tensor_stream = st.TensorStream(f,idxtypes, sep=',', mappers={})

### Class TensorStream
**Input**

**f:**: file iterator of input file

**idxtypes**: column idx-type dict

**sep**: the delimeter of input file

**mappers**: dict of idx-mapper dict, we have defined various mappers.

StringMapper: mapping the names or complex string ids of users and objects into indices. 

ScoreMapper: mapping score (int or float) into index (int).

TimeMapper: mapping the time(string) with some formats into timestamps (int).

**Return**

an instance of TensorStream class.

### Run EigenPulse as a single model

In [ ]:
param_dict={'window':10, 'stride':5, 'l':20, 'b':10,'item_idx':1,'ts_idx':2, 'hasvalue':True}
eigenpulse = st.EigenPulse(tensor_stream, **param_dict)

### the parameters of EigenPulse model

**window**: the window size of sliding window in a time unit

**stride**: the stride size in a time unit.

**l and b**: hyper-parameters of Algorithm 1.

**item_idx**: the column index of item attribute. 

**ts_idx**: the column index of time attribute.

EigenPulse first concentrates items by the time as row, user as column, and thus the row of modified tensor grows with the forward of time. We model the streaming tuples as row-augmented matrix in this way.

**hasvalue**: Whether "tensor_data.data" contains a single-colum values (optional) or not.

In [ ]:
eigenpulse.run()

### Run EigenPulse from anomaly detection task

In [ ]:
ad_model = st.AnomalyDetection.create(tensor_stream, st.ADPolicy.EigenPulse, 'eigenpulse',**param_dict)

In [ ]:
ad_model.run()

The results is a list of suspicious blocks whose densities are above the threshold ($\mu+3\sigma$). For each block, the results are the lists of suspicious users, objects and times.

### Experimental results:
-----
EigenPulse (performance)       |  EigenPulse (inject attacks)
:-------------------------:|:-------------------------:
<img src="images/eigen_performance.png" width="300"/>  |   <img src="images/eigeninject.png" width="300"/>
<b>EigenPulse detection on real Sina Weibo data |  <b>EigenPulse is near linear
<img src="images/eigenweibo.png" width="300"/> |   <img src="images/eigenlinear.png" width="300"/>


### Cite:
Zhang J, Liu S, Yu W, et al. Eigenpulse: Detecting surges in large streaming graphs with row augmentation[C]//Pacific-Asia Conference on Knowledge Discovery and Data Mining. Springer, Cham, 2019: 501-513.